In [23]:
def cal_result(exp_name, criterion='best'):
    import dill as pickle
    data_stream = 70
    week_n = 7000 

    with open('data/y.pkl', 'rb') as f:
        y = pickle.load(f).flatten().tolist()

    for stream in range(1, data_stream + 1): 
        pre = (stream - 1) * 7000
        
        flag = False 
        start, end = pre + week_n, pre + 2 * week_n
        for j in range(start, end): 
            if y[j] == 1: 
                flag = True; break 

        def read_score(exp_name, week): 
            lst = []
            exp_name = os.path.join(os.path.join("exp", exp_name), exp_name)
            with open(exp_name + "Week" + str(week) +  "_result.txt", "r") as f: 
                sc = float(f.readline()) 
                while sc != None: 
                    lst.append(sc)

                    s = f.readline()
                    # while !s: s = f.readline()
                    if s == '': break  
                    sc = float(s) 

            return lst                

        if flag : 
            x_score = read_score(exp_name, stream)
            from sklearn.metrics import roc_curve, auc
            GT = y[start: end]
            from sklearn.metrics import roc_auc_score
            AUC = roc_auc_score(GT, x_score)
            print(stream, " week :", AUC)  

            # print(len(x_score), GT) 
            FPR, TPR, thresholds = roc_curve(GT, x_score)
            maxindex = (TPR-FPR).tolist().index(max(TPR-FPR))
            threshold = thresholds[maxindex]
            tpr, fpr = TPR[maxindex], FPR[maxindex]
            print("tpr: %f , fpr: %f, thresh: %f\n" % (tpr, fpr, threshold))
            # if criterion == 'bset': 
                # JD = [x - y for x, y in zip(TPR, FPR)]
            


In [24]:
cal_result("exp_all_data_finetune")

22  week : 0.9987139182623607
tpr: 1.000000 , fpr: 0.002572, thresh: 2.000216

23  week : 0.8301859799713877
tpr: 0.800000 , fpr: 0.217167, thresh: 0.760091

24  week : 0.8859651634454784
tpr: 0.800000 , fpr: 0.129563, thresh: 0.814887

25  week : 0.8046131805157593
tpr: 0.800000 , fpr: 0.256017, thresh: 0.694942

26  week : 0.8088028673835126
tpr: 0.800000 , fpr: 0.244588, thresh: 0.673488

27  week : 0.829725155523028
tpr: 0.681818 , fpr: 0.169350, thresh: 0.712724

28  week : 0.8416857289145754
tpr: 0.743590 , fpr: 0.188191, thresh: 0.691572

29  week : 0.8089686495088018
tpr: 0.794872 , fpr: 0.282574, thresh: 0.612135

30  week : 0.8352803898219248
tpr: 0.857143 , fpr: 0.302817, thresh: 0.626570

31  week : 0.8358709289560353
tpr: 0.772727 , fpr: 0.221392, thresh: 0.644567

32  week : 0.7728694404591105
tpr: 0.800000 , fpr: 0.305452, thresh: 0.576184

33  week : 0.8436228352786486
tpr: 0.826087 , fpr: 0.225025, thresh: 0.604359

34  week : 0.8534996415770609
tpr: 0.880000 , fpr: 0.